# Narrow with SB3's DQN

##  Warming up
We start with a few useful installs and imports:

In [1]:
# Install environment and agent
!pip install git+https://github.com/liuxin00738/highway-env.git
!pip install stable-baselines3

# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import DQN

# Visualization utils
%load_ext tensorboard
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!git clone https://github.com/liuxin00738/highway-env.git 2> /dev/null
sys.path.insert(0, '/content/highway-env/scripts/')
from utils import record_videos, show_videos

  Cloning https://github.com/liuxin00738/highway-env.git to /tmp/pip-req-build-hwy2yxkf
  Running command git clone -q https://github.com/liuxin00738/highway-env.git /tmp/pip-req-build-hwy2yxkf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 21.8 MB 1.2 MB/s 
  Created wheel for highway-env: filename=highway_env-0.0.0-py3-none-any.whl size=100307 sha256=ec97c7457e9d0c548802235088fc071a7a2281558bf7aaf299a7ddaeea1e407d
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ncygx7o/wheels/a3/73/b7/6dd9ff57227db6ece3aa04e53c50d136a9cf249b4aa48aeeca
Successfully built highway-env
     |████████████████████████████████| 174 kB 5.2 MB/s 
     |████████████████████████████████| 124 kB 5.1 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubun

# Playground

Play with some environments

In [ ]:
import pprint
from matplotlib import pyplot as plt
%matplotlib inline

env = gym.make("narrow-v0")
config = {
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 3,
        "features": ["presence", "x", "y", "vx", "vy"],
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-20, 20],
            "vy": [-20, 20]
        },
        "absolute": False,
        "order": "sorted",
        "normalize": False,
    }
    "other_vehicles_type": "highway_env.vehicle.kinematics.Vehicle",
}
env.configure(config)
pprint.pprint(env.config)

# TODO(xin): set corrent inputs and number of cars

# Render an image and show it.
env.reset()
for i in range(1):
    action = env.action_type.actions_indexes["IDLE"]
    obs, reward, done, info = env.step(action)
    print("obs at %d is:\n"%i, obs)
    env.render()

plt.figure(figsize = (25,10))
plt.imshow(env.render(mode="rgb_array"))
plt.show()

# # Record a video
# env = record_videos(env)
# obs, done = env.reset(), False
# while not done:
#     action = env.action_type.actions_indexes["LANE_LEFT"]
#     obs, reward, done, info = env.step(action)
# env.close()
# show_videos()

## Training
Run tensorboard locally to visualize training.

In [ ]:
%tensorboard --logdir "highway_dqn"

In [ ]:
model = DQN('MlpPolicy', "highway-fast-v0",
                policy_kwargs=dict(net_arch=[256, 256]),
                learning_rate=5e-4,
                buffer_size=15000,
                learning_starts=200,
                batch_size=32,
                gamma=0.8,
                train_freq=1,
                gradient_steps=1,
                target_update_interval=50,
                exploration_fraction=0.7,
                verbose=1,
                tensorboard_log="highway_dqn/")
model.learn(int(2e4))


## Testing

Visualize a few episodes

In [ ]:
env = gym.make("highway-fast-v0")
env = record_videos(env)
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
env.close()
show_videos()
